In [2]:
import pandas as pd

df = pd.read_csv(
    r'C:\Users\maxpi\Downloads\covid-hospit-incid-reg-2023-03-31-18h01.csv',
    encoding='latin1',
    sep=';',  
    parse_dates=['jour']
)

print(df.head())

        jour      nomReg  numReg  incid_rea
0 2020-03-19  Guadeloupe     1.0          0
1 2020-03-20  Guadeloupe     1.0          0
2 2020-03-21  Guadeloupe     1.0          0
3 2020-03-22  Guadeloupe     1.0          5
4 2020-03-23  Guadeloupe     1.0          0


In [3]:
print(df.columns.tolist())
print(df.dtypes)



['jour', 'nomReg', 'numReg', 'incid_rea']
jour         datetime64[ns]
nomReg               object
numReg              float64
incid_rea             int64
dtype: object


In [4]:
print(df.isnull().sum())
empty_rows = df[df.isnull().all(axis=1)]
print(empty_rows)
missing_data = df[df.isnull().any(axis=1)]
print(missing_data.head())

jour            0
nomReg       1108
numReg       1108
incid_rea       0
dtype: int64
Empty DataFrame
Columns: [jour, nomReg, numReg, incid_rea]
Index: []
            jour nomReg  numReg  incid_rea
19944 2020-03-19    NaN     NaN          0
19945 2020-03-20    NaN     NaN          0
19946 2020-03-21    NaN     NaN          0
19947 2020-03-22    NaN     NaN          0
19948 2020-03-23    NaN     NaN          0


In [5]:
# First I need to clean the columns - rename column titles.
# jour = we need to split into day month and year
# nomReg = region
# numReg = delete - it is just a registry number of the region
# incid_rea = number_of_cases

# As we are checking for per month, I will need to sum each number of cases per month

In [6]:
df = df.rename(columns={
    'jour': 'day',
    'nomReg': 'region',
    'incid_rea': 'number_of_cases'
})

overseas = ['Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte']

df = df[~df['region'].isin(overseas)].reset_index(drop=True)

df = df.drop(columns=['numReg'])

df['year'] = df['day'].dt.year
df['month'] = df['day'].dt.month

# Group by region, year, and month and sum the cases
monthly_cases = df.groupby(['region', 'year', 'month'])['number_of_cases'].sum().reset_index()

print(monthly_cases.head())


                 region  year  month  number_of_cases
0  Auvergne-Rhône-Alpes  2020      3              796
1  Auvergne-Rhône-Alpes  2020      4              847
2  Auvergne-Rhône-Alpes  2020      5              156
3  Auvergne-Rhône-Alpes  2020      6               28
4  Auvergne-Rhône-Alpes  2020      7               25


In [7]:
# checking for nulls and cleaning them
print(df.isnull().sum())
empty_rows = df[df.isnull().all(axis=1)]
print(empty_rows)
missing_data = df[df.isnull().any(axis=1)]
print(missing_data.head())
missing_region_rows = df[df['region'].isnull()]
print(missing_region_rows.head(20)) 
# Check rows have 0 cases
all_zero = (missing_region_rows['number_of_cases'] == 0).all()
print(all_zero)
# All null regions have 0 cases, so I will delete them all as they will not contribute to the analysis
df = df[df['region'].notna()]
print(df['region'].isnull().sum())

day                   0
region             1108
number_of_cases       0
year                  0
month                 0
dtype: int64
Empty DataFrame
Columns: [day, region, number_of_cases, year, month]
Index: []
             day region  number_of_cases  year  month
14404 2020-03-19    NaN                0  2020      3
14405 2020-03-20    NaN                0  2020      3
14406 2020-03-21    NaN                0  2020      3
14407 2020-03-22    NaN                0  2020      3
14408 2020-03-23    NaN                0  2020      3
             day region  number_of_cases  year  month
14404 2020-03-19    NaN                0  2020      3
14405 2020-03-20    NaN                0  2020      3
14406 2020-03-21    NaN                0  2020      3
14407 2020-03-22    NaN                0  2020      3
14408 2020-03-23    NaN                0  2020      3
14409 2020-03-24    NaN                0  2020      3
14410 2020-03-25    NaN                0  2020      3
14411 2020-03-26    NaN         

In [8]:



print(df['region'].unique())


['Ile-de-France' 'Centre-Val de Loire' 'Bourgogne-Franche-Comté'
 'Normandie' 'Hauts-de-France' 'Grand-Est' 'Pays de la Loire' 'Bretagne'
 'Nouvelle-Aquitaine' 'Occitanie' 'Auvergne-Rhône-Alpes'
 "Provence-Alpes-Côte d'Azur" 'Corse']


In [9]:
#function to view each region and see the month by month case change in the year 2022
df['change_in_cases'] = df.groupby(['region', 'year'])['number_of_cases'].diff().fillna(0)

def view_region_monthly_change(region_name):
    # Filter for the selected region and year
    region_data = monthly_cases[
        (monthly_cases['region'] == region_name)
    ].sort_values('month')
    
    region_data['change_in_cases'] = region_data['number_of_cases'].diff().fillna(0)
    
    return region_data.reset_index(drop=True)


In [10]:
print(view_region_monthly_change('Ile-de-France'))


           region  year  month  number_of_cases  change_in_cases
0   Ile-de-France  2023      1              206              0.0
1   Ile-de-France  2022      1             2425           2219.0
2   Ile-de-France  2021      1             1364          -1061.0
3   Ile-de-France  2022      2             1073           -291.0
4   Ile-de-France  2023      2              213           -860.0
5   Ile-de-France  2021      2             1731           1518.0
6   Ile-de-France  2020      3             2372            641.0
7   Ile-de-France  2022      3              582          -1790.0
8   Ile-de-France  2021      3             3560           2978.0
9   Ile-de-France  2023      3              283          -3277.0
10  Ile-de-France  2020      4             3787           3504.0
11  Ile-de-France  2021      4             3963            176.0
12  Ile-de-France  2022      4              750          -3213.0
13  Ile-de-France  2021      5             1728            978.0
14  Ile-de-France  2022  

In [11]:
# Sort by the largest monthly_change
top_monthly_changes = df.sort_values('change_in_cases', ascending=False)



In [12]:
print(top_monthly_changes.head(10))


             day                region  number_of_cases  year  month  \
11028 2023-02-08             Occitanie              104  2023      2   
397   2021-04-20         Ile-de-France              208  2021      4   
12    2020-03-31         Ile-de-France              288  2020      3   
383   2021-04-06         Ile-de-France              156  2021      4   
5     2020-03-24         Ile-de-France              236  2020      3   
11767 2022-02-04  Auvergne-Rhône-Alpes              104  2022      2   
11476 2021-04-19  Auvergne-Rhône-Alpes               85  2021      4   
648   2021-12-27         Ile-de-France               79  2021     12   
11469 2021-04-12  Auvergne-Rhône-Alpes               87  2021      4   
662   2022-01-10         Ile-de-France              105  2022      1   

       change_in_cases  
11028            103.0  
397              102.0  
12                99.0  
383               83.0  
5                 81.0  
11767             75.0  
11476             67.0  
648    

In [14]:
# See which regions had biggest change in cases 

monthly_cases['change_in_cases'] = monthly_cases.groupby(['region'])['number_of_cases'].diff().fillna(0)

top_changes_all_years = monthly_cases.sort_values('change_in_cases', ascending=False)

print(top_changes_all_years.head(10))

                         region  year  month  number_of_cases  change_in_cases
271               Ile-de-France  2021      3             3560           1829.0
260               Ile-de-France  2020      4             3787           1415.0
266               Ile-de-France  2020     10             1851           1110.0
280               Ile-de-France  2021     12             1667           1031.0
465  Provence-Alpes-Côte d'Azur  2021     12             1162            888.0
7          Auvergne-Rhône-Alpes  2020     10             1134            834.0
8          Auvergne-Rhône-Alpes  2020     11             1892            758.0
281               Ile-de-France  2022      1             2425            758.0
21         Auvergne-Rhône-Alpes  2021     12             1102            723.0
461  Provence-Alpes-Côte d'Azur  2021      8              882            681.0


In [15]:
# to confirm, I will check the sum of the absolute change 
import numpy as np

monthly_cases['change_in_cases'] = monthly_cases.groupby('region')['number_of_cases'].diff().fillna(0)

region_abs_change = monthly_cases.groupby('region')['change_in_cases'].apply(lambda x: x.abs().sum()).reset_index()

region_abs_change = region_abs_change.sort_values('change_in_cases', ascending=False)

print(region_abs_change)

                        region  change_in_cases
7                Ile-de-France          21591.0
0         Auvergne-Rhône-Alpes          10178.0
12  Provence-Alpes-Côte d'Azur           8575.0
6              Hauts-de-France           7337.0
10                   Occitanie           7332.0
5                    Grand-Est           5951.0
9           Nouvelle-Aquitaine           4114.0
1      Bourgogne-Franche-Comté           3204.0
3          Centre-Val de Loire           2726.0
11            Pays de la Loire           2493.0
8                    Normandie           2365.0
2                     Bretagne           1867.0
4                        Corse            490.0


In [ ]:
region_abs_change_sorted = region_abs_change.sort_values('sum_abs_change', ascending=False)
print(region_abs_change_sorted)








                        region  sum_abs_change
7                Ile-de-France          3772.0
10                   Occitanie          1801.0
0         Auvergne-Rhône-Alpes          1790.0
12  Provence-Alpes-Côte d'Azur          1622.0
6              Hauts-de-France          1333.0
5                    Grand-Est          1175.0
9           Nouvelle-Aquitaine          1053.0
1      Bourgogne-Franche-Comté           762.0
11            Pays de la Loire           665.0
3          Centre-Val de Loire           635.0
2                     Bretagne           609.0
8                    Normandie           434.0
4                        Corse           143.0


In [ ]:
#We can focus our air quality analysis in Ile-de-France, Occitanie, Provence-Alpes-Côte d'Azur

In [ ]:
df

,day,region,number_of_cases,year,month,change_in_cases
0,2020-03-19,Ile-de-France,151,2020,3,0.0
1,2020-03-20,Ile-de-France,89,2020,3,-62.0
2,2020-03-21,Ile-de-France,104,2020,3,15.0
3,2020-03-22,Ile-de-France,125,2020,3,21.0
4,2020-03-23,Ile-de-France,155,2020,3,30.0
...,...,...,...,...,...,...
14399,2023-03-27,Corse,0,2023,3,0.0
14400,2023-03-28,Corse,0,2023,3,0.0
14401,2023-03-29,Corse,0,2023,3,0.0
14402,2023-03-30,Corse,0,2023,3,0.0


In [ ]:
print(df.columns.tolist())


['date', 'region', 'number_of_cases', 'year', 'month', 'change_in_cases']


In [ ]:
print(df['region'].unique())


['Ile-de-France' 'Centre-Val de Loire' 'Bourgogne-Franche-Comté'
 'Normandie' 'Hauts-de-France' 'Grand-Est' 'Pays de la Loire' 'Bretagne'
 'Nouvelle-Aquitaine' 'Occitanie' 'Auvergne-Rhône-Alpes'
 "Provence-Alpes-Côte d'Azur" 'Corse']


In [ ]:
# We need to merge this dataset with the air quality dataset, and then we can see how air quality has changed with covid cases.

In [ ]:
df

,date,region,number_of_cases,year,month,change_in_cases
0,2020-03-19,Ile-de-France,151,2020,3,0.0
1,2020-03-20,Ile-de-France,89,2020,3,-62.0
2,2020-03-21,Ile-de-France,104,2020,3,15.0
3,2020-03-22,Ile-de-France,125,2020,3,21.0
4,2020-03-23,Ile-de-France,155,2020,3,30.0
...,...,...,...,...,...,...
14399,2023-03-27,Corse,0,2023,3,0.0
14400,2023-03-28,Corse,0,2023,3,0.0
14401,2023-03-29,Corse,0,2023,3,0.0
14402,2023-03-30,Corse,0,2023,3,0.0


In [ ]:
# Check for duplicates
duplicates = df.duplicated()
num_duplicates = duplicates.sum()
print(df[duplicates])


Empty DataFrame
Columns: [date, region, number_of_cases, year, month, change_in_cases]
Index: []


In [ ]:
#change column order for better visibility

cols = ['region','day', 'month', 'year','number_of_cases','change_in_cases']
new_order = cols 
df = df[new_order]

In [ ]:
df

,region,day,month,year,number_of_cases,change_in_cases
0,Ile-de-France,2020-03-19,3,2020,151,0.0
1,Ile-de-France,2020-03-20,3,2020,89,-62.0
2,Ile-de-France,2020-03-21,3,2020,104,15.0
3,Ile-de-France,2020-03-22,3,2020,125,21.0
4,Ile-de-France,2020-03-23,3,2020,155,30.0
...,...,...,...,...,...,...
14399,Corse,2023-03-27,3,2023,0,0.0
14400,Corse,2023-03-28,3,2023,0,0.0
14401,Corse,2023-03-29,3,2023,0,0.0
14402,Corse,2023-03-30,3,2023,0,0.0
